# Apptainerの利用

---

構築したOpenHPC環境で[Apptainer](https://apptainer.org/)を利用します。

## 前提条件

このNotebookを実行するための前提条件を満たしていることを確認します。

以下のことを前提条件とします。
* 構築済のOpenHPC環境がある
* OpenHPC環境のマスターノードに対してSSHでログインできる

> この Notebookでは、管理ユーザ以外が実行した状況に合わせるために Ansible ではなく ssh で操作を行います。

マスターノードに対して SSH でログインできることを確認します。マスターノードのアドレスを指定してください。

In [ ]:
# (例)
# master_address = '172.30.XXX.xxx'

master_address = 

SSHでログインするユーザ名を指定してください。

In [ ]:
# (例)
# user = 'vcp'
# user = 'user00'

user = 

必要であればSSHの秘密鍵を指定してください。

In [ ]:
# (例)
# ssh_identity = '~/.ssh/id_rsa'

SSHでログインする際の引数を、変数に格納しておきます。

In [ ]:
# ユーザ名とホスト名
target = f'{user}@{master_address}'
print(target)

# SSHのコマンドライン引数
ssh_opts = f'-i {ssh_identity}' if 'ssh_identity' in vars() else ''
print(ssh_opts)

マスターノードに対してSSHでログインしてコマンドを実行してみます。

In [ ]:
!ssh {ssh_opts} {target} hostname

## Apptainerの実行

apptainerで docker コンテナイメージを実行してみます。

> apptainerでの操作の詳細については Apptainerの[User Guide](https://apptainer.org/docs/user/1.3/)などを参照してください。

dockerhub にある RockyLinux のコンテナイメージを apptainer で実行してみます。まずコンテナイメージを取得してみます。

In [ ]:
!ssh {ssh_opts} {target} bash -l -c \
    "'test -f rockylinux_8.sif || apptainer pull docker://rockylinux:8'"

コンテナを実行して `/etc/redhat-release` の内容を表示してみます。

In [ ]:
!ssh {ssh_opts} {target} bash -l -c \
    "'apptainer exec rockylinux_8.sif cat /etc/redhat-release'"

SLURMで実行してみます。

まず、実行するノード数を指定してください。

In [ ]:
# (例)
# num_nodes = 2

num_nodes = 

srunで実行します。

In [ ]:
!ssh {ssh_opts} {target} bash -l -c \
    "'srun -l -N {num_nodes} \
    --deadline=$(date -d "3 minutes" "+%H:%M") \
    apptainer exec rockylinux_8.sif cat /etc/redhat-release'"